# DGILib

This chapter will instantiate the connection to the Atmel SAML11 via the DGILib API.

The API functions are separated into four groups:
- Discovery: Used to discover available devices and get information about them.
- Housekeeping: Provides version information, connection, and session control.
- Interface: Communication Handles communication with the various interfaces of DGI.
- Auxiliary: Extended functionality with interface-specifc usage.

## Discovery

Import the `ctypes` library.

In [59]:
from ctypes import *
import typing
import time

In [60]:
dgilib = cdll.LoadLibrary("dgilib")

Get the build information of DGILib.

In [61]:
major_version = dgilib.get_major_version()
minor_version = dgilib.get_minor_version()
build_number = dgilib.get_build_number()

In [62]:
print("major_version: {0}, minor_version: {1}, build_number: {2}".format(major_version, minor_version, build_number))

major_version: 5, minor_version: 7, build_number: 244


In [63]:
GET_STRING_SIZE = 100
NUM_INTERFACES = 10
NUM_CONFIG_IDS = 10
NUM_CALIBRATION = 255
BUFFER_SIZE = 10000000

INTERFACE_TIMESTAMP  = 0x00 # Service interface which appends timestamps to all received events on associated interfaces.
INTERFACE_SPI        = 0x20 # Communicates directly over SPI in Slave mode.
INTERFACE_USART      = 0x21 # Communicates directly over USART in Slave mode.
INTERFACE_I2C        = 0x22 # Communicates directly over I2C in Slave mode.
INTERFACE_GPIO       = 0x30 # Monitors and controls the state of GPIO pins.
INTERFACE_POWER_DATA = 0x40 # Receives data from the attached power measurement co-processors.
INTERFACE_POWER_SYNC = 0x41 # Receives sync events from the attached power measurement co-processors.
INTERFACE_RESERVED   = 0xFF # Special identifier used to indicate no interface.

#### `discover`
Triggers a scan to find available devices in the system. The result will be immediately available through the `get_device_count`, `get_device_name` and `get_device_serial` functions.

`void discover(void)`

In [64]:
dgilib.discover()

2

#### `get_device_count`
Returns the number of devices detected.

`int get_device_count(void)`

In [65]:
device_count = dgilib.get_device_count()
print("device_count: {0}".format(device_count))

device_count: 1


Select device to use. Defaults to `device_count - 1`.

In [66]:
device_index = device_count - 1

#### `get_device_name`
Gets the name of a detected device.

A non-zero return value indicates an error.

`int get_device_name(int index, char* name)`

Parameter   | Description
-------------  | -------------
*index* | Index of device ranges from 0 to `get_device_count` - 1
*name* | Pointer to buffer where name of device can be stored. 100 or more bytes must be allocated

In [67]:
name = create_string_buffer(GET_STRING_SIZE)
res = dgilib.get_device_name(device_index, byref(name))
print("{0} get_device_name: {1}".format(res, repr(name.value)))

0 get_device_name: b'SAM L11 Xplained Pro'


#### `get_device_serial`
Gets the serial number of a detected device.

A non-zero return value indicates an error.

`int get_device_serial(int index, char* sn)`

Parameter   | Description
-------------  | -------------
*index* | Index of device ranges from 0 to `get_device_count` - 1
*sn* | Pointer to buffer where the serial number of the device can be stored. 100 or more bytes must be allocated. This is used when connecting to a device.

In [68]:
sn = create_string_buffer(GET_STRING_SIZE)
res = dgilib.get_device_serial(device_index, byref(sn))
print("{0} get_device_serial: {1}".format(res, repr(sn.value)))

0 get_device_serial: b'ATML3138061800001604'


In [69]:
# Initialize (not in manual, exists in dgilib.h)
dgi_hndl = c_uint()
dgilib.Initialize(byref(dgi_hndl))

2

#### `is_msd_mode`
EDBG devices can be set to a mass storage mode where the DGI is unavailable. In such cases the device is still detected by DGILib, but it won't be possible to directly connect to it. This command is used to check if the device is in such a mode.

A non-zero return value indicates that the mode must be changed by `set_mode` before proceeding.

`int is_msd_mode(char* sn)`

Parameter   | Description
-------------  | -------------
*sn* | Serial number of the device to check

#### `set_mode`
This function is used to temporarily set the EDBG to a specified mode.

A non-zero return value indicates an error.

`int set_mode(char* sn, int nmbed)`

Parameter   | Description
-------------  | -------------
*sn* | Serial number of the device to check
*nmbed* | 0 - Set to mbed mode. 1 - Set to DGI mode.

Check and correct `msd_mode` to make sure the device is in DGI mode.

In [70]:
msd_mode = dgilib.is_msd_mode(sn)
# if msd_mode:
#     res = dgilib.set_mode(sn, 1)
#     print("{0} set_mode 1".format(res))
print("msd_mode: {0}".format(msd_mode))

msd_mode: 0


## Housekeeping

#### `connect`
Opens a connection to the specified device. This function must be called prior to any function requiring the connection handle.

A non-zero return value indicates an error.

`int connect(char* sn, uint32_t* dgi_hndl_p)`

Parameter   | Description
-------------  | -------------
*sn* | Buffer holding the serial number of the device to open a connection to
*dgi_hndl_p* | Pointer to a variable that will hold the handle of the connection

In [71]:
res = dgilib.connect(sn, byref(dgi_hndl))
print("{0} connect".format(res))

0 connect


#### `connection_status`
Verifies that the specified connection is still open.

A non-zero return value indicates an error.

`int connection_status(uint32_t* dgi_hndl)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle of the connection

In [72]:
c_status = dgilib.connection_status(dgi_hndl)
print("connection_status: {0}".format(c_status))

connection_status: 0


#### `target_reset`
This function is used to control the state of the reset line connected to the target, if available.

A non-zero return value indicates an error.

`int target_reset(uint32_t dgi_hndl, bool hold_reset)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle of the connection
*hold_reset* | True will assert reset, false will release it

In [73]:
print("Resetting {0}".format(repr(name.value)))
print(dgilib.target_reset(dgi_hndl, True))
time.sleep(1)
print(dgilib.target_reset(dgi_hndl, False))
print("Done");

Resetting b'SAM L11 Xplained Pro'
0
0
Done


In [74]:
c_status = dgilib.connection_status(dgi_hndl)
print("connection_status: {0}".format(c_status))

connection_status: 0


#### `get_fw_version`
Gets the firmware version of the DGI device connected. Note that this is the version of the DGI device, and not the tool.

A non-zero return value indicates an error.

`int get_fw_version(uint32_t dgi_hndl, unsigned char* major, unsigned char* minor)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle of the connection
*major* | Pointer to a variable where the major version will be stored
*minor* | Pointer to a variable where the minor version will be stored

In [75]:
major = c_ubyte()
minor = c_ubyte()
res = dgilib.get_fw_version(dgi_hndl, byref(major), byref(minor))
print("{0} fw_version major: {1}, minor: {2}".format(res, repr(major.value), repr(minor.value)))

0 fw_version major: 3, minor: 2


## Interface

#### `interface_list`
Query the connected DGI device for available interfaces. Refer to the DGI documentation to resolve the ID.

`int interface_list(uint32_t dgi_hndl, unsigned char* interfaces, unsigned char* count)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interfaces* | Buffer to hold the ID of the available interfaces. Should be able to hold minimum 10 elements, but a larger count should be used to be future proof.
*count* | Pointer to a variable that will be set to the number of interfaces registered in buffer.

In [76]:
interfaces = create_string_buffer(NUM_INTERFACES)
interfaceCount = c_ubyte()

In [77]:
res = dgilib.interface_list(dgi_hndl, byref(interfaces), byref(interfaceCount))
print("{0} interface_list: {1}, interfaceCount: {2}".format(res, interfaces[:], repr(interfaceCount.value)))

0 interface_list: b'\x00 "0@AP\x00\x00\x00', interfaceCount: 7


#### `interface_get_configuration`
Gets the configuration associated with the specified interface. Consult the DGI documentation for details.

A non-zero return value indicates an error.

`int interface_get_configuration(uint32_t dgi_hndl, int interface_id, unsigned int* config_id, unsigned int* config_value, unsigned int* config_cnt)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface
*config_id* | Buffer that will hold the ID field for the configuration item
*config_value* | Buffer that will hold the value field for the configuration item
*config_cnt* | Pointer to variable that will hold the count of stored configuration items

In [78]:
config_id = (c_uint * NUM_CONFIG_IDS)()
config_value = (c_uint * NUM_CONFIG_IDS)()
config_cnt = c_uint()
res = dgilib.interface_get_configuration(dgi_hndl, INTERFACE_TIMESTAMP, byref(config_id), byref(config_value), byref(config_cnt))
print("{0} interface_get_configuration: {1}, config_cnt: {2}".format(res, INTERFACE_TIMESTAMP, repr(config_cnt.value)))
for i in range(config_cnt.value):
    print("\tconfig_id: {0},\tvalue: {1}".format(config_id[i], config_value[i]))

0 interface_get_configuration: 0, config_cnt: 2
	config_id: 0,	value: 32
	config_id: 1,	value: 60000000


In [79]:
timerPrescaler = config_value[0]
timerFrequency = config_value[1]
timerFactor = timerPrescaler / timerFrequency
print("timerFactor: {0}".format(timerFactor))

timerFactor: 5.333333333333333e-07


#### `interface_enable`
Enables the specified interface. Note that no data acquisition will begin until a session has been started.

A non-zero return value indicates an error.

`int interface_enable(uint32_t dgi_hndl, int interface_id, bool timestamp)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface to enable
*timestamp* | Setting this to true will make the interface use timestamping. Consult the DGI documentation for details on the timestamping option.

In [80]:
interface_id = INTERFACE_GPIO;
enableTimestamp = True;
res = dgilib.interface_enable(dgi_hndl, interface_id, enableTimestamp);
print("{0} interface_enable: {1}, enableTimestamp: {2}\n".format(res, interface_id, enableTimestamp))

0 interface_enable: 48, enableTimestamp: True



In [81]:
for i in range(NUM_CONFIG_IDS):
    config_id[i] = 0
    config_value[i] = 0
res = dgilib.interface_get_configuration(dgi_hndl, interface_id, byref(config_id), byref(config_value), byref(config_cnt))
print("{0} interface_get_configuration: {1}, config_cnt: {2}".format(res, interface_id, repr(config_cnt.value)))
for i in range(config_cnt.value):
    print("\tconfig_id: {0},\tvalue: {1}".format(config_id[i], config_value[i]))

0 interface_get_configuration: 48, config_cnt: 2
	config_id: 0,	value: 0
	config_id: 1,	value: 0


#### `interface_set_configuration`
Sets the given configuration fields for the specified interface. Consult the DGI documentation for details.

A non-zero return value indicates an error.

`int interface_set_configuration(uint32_t dgi_hndl, int interface_id, unsigned int* config_id, unsigned int*
config_value, unsigned int config_cnt)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface
*config_id* | Buffer that holds the ID field for the configuration items to set
*config_value* | Buffer that holds the value field for the configuration items to set
*config_cnt* | Number of items to set

In [82]:
config_value[0] = 0b1111
config_value[1] = 0b0000
res = dgilib.interface_set_configuration(dgi_hndl, interface_id, byref(config_id), byref(config_value), config_cnt)
print("{0} interface_set_configuration: {1}, config_cnt: {2}".format(res, interface_id, repr(config_cnt.value)))
for i in range(config_cnt.value):
    print("\tconfig_id: {0},\tvalue: {1}".format(config_id[i], config_value[i]))

0 interface_set_configuration: 48, config_cnt: 2
	config_id: 0,	value: 15
	config_id: 1,	value: 0


#### `interface_clear_buffer`
Clears the data in the buffers for the specified interface.

A non-zero return value indicates an error.

`int interface_clear_buffer(uint32_t dgi_hndl, int interface_id)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface

In [83]:
res = dgilib.interface_clear_buffer(dgi_hndl, interface_id)
print("{0} interface_clear_buffer: {1}".format(res, interface_id))

0 interface_clear_buffer: 48


#### `start_polling`
This function will start the polling system and start acquisition on enabled interfaces. It is possible to enable/disable interfaces both before and after the polling has been started. However, no data will be transferred until the polling is started.

A non-zero return value indicates an error.

`int start_polling(uint32_t dgi_hndl)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle of the connection

In [84]:
res = dgilib.start_polling(dgi_hndl)
print("{0} start_polling".format(res))

0 start_polling


#### `interface_read_data`
Reads the data received on the specified interface. This should be called regularly to avoid overflows in the system. DGILib can buffer 10M samples.

A non-zero return value indicates an error.

`int interface_read_data(uint32_t dgi_hndl, int interface_id, unsigned char* buffer, unsigned long long* timestamp, int* length, unsigned int* ovf_index, unsigned int* ovf_length, unsigned int* ovf_entry_count)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface
*buffer* | Buffer that will hold the received data. The buffer must have allocated 10M elements.
*timestamp* | If timestamp is enabled for the interface, the buffer that will hold the received data. The buffer must have allocated 10M elements. Otherwise send 0.
*length* | Pointer to a variable that will hold the count of elements received
*ovf_index* | Reserved. Set to 0.
*ovf_length* | Reserved. Set to 0.
*ovf_entry_count* | Reserved. Set to 0. Could be set to a pointer to a variable that can be used as an indicator of overflows. Overflow would be indicated by non-zero value.

In [87]:
# readBuffer = create_string_buffer(BUFFER_SIZE)
readBuffer = (c_ubyte * BUFFER_SIZE)()
timestamp = (c_ulonglong * BUFFER_SIZE)()
length = c_uint(0)
ovf_index = c_uint(0)
ovf_length = c_uint(0)
ovf_entry_count = c_uint(0)
res = dgilib.interface_read_data(dgi_hndl, interface_id, readBuffer, timestamp, byref(length), byref(ovf_index), byref(ovf_length), byref(ovf_entry_count));
print("{0} interface_read_data: {1}, length: {2}".format(res, interface_id, length.value))
for i in range(length.value):
    print("\t{0}: buffer: {1}, timestamp: {2}, time: {3}".format(i, readBuffer[i], timestamp[i], timestamp[i] * timerFactor))

0 interface_read_data: 48, length: 0


#### `interface_write_data`
Writes data to the specified interface. A maximum of 255 elements can be written each time. An error return code will be given if data hasn't been written yet.

A non-zero return value indicates an error. An error will be returned if the interface is still in the process of writing data. Wait a while and try again. The function get_connection_status can be used to verify if there is an error condition.

`int interface_write_data(uint32_t dgi_hndl, int interface_id, unsigned char* buffer, int* length)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface
*buffer* | Buffer that will hold the received data. The buffer must have allocated 10M elements.
*length* | Pointer to a variable that will hold the count of elements received

In [88]:
# writeBuffer = (c_ubyte * NUM_CONFIG_IDS)()
# writeLength = c_int(2)
# res = dgilib.interface_write_data(dgi_hndl, interface_id, byref(writeBuffer), byref(writeLength))
# print("{0} interface_write_data: {1}, length: {2}".format(res, interface_id, writeLength.value))

In [89]:
# res = dgilib.interface_read_data(dgi_hndl, interface_id, readBuffer, timestamp, byref(length), byref(ovf_index), byref(ovf_length), byref(ovf_entry_count));
# print("{0} interface_read_data: {1}, length: {2}".format(res, interface_id, length.value))
# for i in range(length.value):
#     print("\t{0}: buffer: {1}, timestamp: {2}, time: {3}".format(i, readBuffer[i], timestamp[i], timestamp[i] * timerFactor))

## Auxiliary

### Power
The power interface (as found on some EDBG kits and Power Debugger) uses a protocol stream and calibration scheme that can be tricky to get right. The data rates are also relatively high and the calibration procedure could cause issues if not handled efficiently. Therefore some auxiliary functions to help with this have been made to perform parsing and calibration.

#### `auxiliary_power_initialize`
Initializes the power parser.

A non-zero return value indicates an error.

`int auxiliary_power_initialize(uint32_t* power_hndl_p, uint32_t dgi_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl_p* | Pointer to variable that will hold the handle to the power parser
*dgi_hndl* | Handle to connection

In [90]:
power_hndl = c_uint()
res = dgilib.auxiliary_power_initialize(byref(power_hndl), dgi_hndl)
print("{0} auxiliary_power_initialize".format(res))

0 auxiliary_power_initialize


#### `auxiliary_power_get_circuit_type`
Gets the type of power circuit.

A non-zero return value indicates an error.

`int auxiliary_power_get_circuit_type(uint32_t power_hndl, int* circuit)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*circuit* | Pointer to a variable that will hold the circuit type: OLD_XAM = 0x00, XAM = 0x10, PAM = 0x11, UNKNOWN = 0xFF

In [91]:
circuit = c_int()
res = dgilib.auxiliary_power_get_circuit_type(power_hndl, byref(circuit))
print("{0} auxiliary_power_get_circuit_type: {1}".format(res, circuit.value))

0 auxiliary_power_get_circuit_type: 16


#### `auxiliary_power_calibration_is_valid`
Checks the status of the stored calibration.

Returns true if the calibration is valid, false otherwise. Unity gain and offset will be used.

`bool auxiliary_power_calibration_is_valid(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser

In [92]:
res = dgilib.auxiliary_power_calibration_is_valid(power_hndl)
print("auxiliary_power_calibration_is_valid: {0}".format(res))

auxiliary_power_calibration_is_valid: 1


#### `auxiliary_power_calibration_is_valid`
Checks the status of the stored calibration.

Returns true if the calibration is valid, false otherwise. Unity gain and offset will be used.

`bool auxiliary_power_calibration_is_valid(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*type* | Type of calibration to trigger. See the DGI documentation for details.

In [93]:
calibrationType = circuit
res = dgilib.auxiliary_power_trigger_calibration(power_hndl, calibrationType)
print("{0} auxiliary_power_trigger_calibration".format(res))
if (not res):
    print("Calibrating {0}".format(name.value))
    while (dgilib.auxiliary_power_get_status(power_hndl) == 0x03):
        print("Calibrating...")
        time.sleep(0.1)
    print("Done")

0 auxiliary_power_trigger_calibration
Calibrating b'SAM L11 Xplained Pro'
Calibrating...
Calibrating...
Done


#### `auxiliary_power_get_status`
Gets the status of the power parser.

Return codes
- `IDLE` = 0x00
- `RUNNING` = 0x01
- `DONE` = 0x02
- `CALIBRATING` = 0x03
- `INIT_FAILED` = 0x10
- `OVERFLOWED` = 0x11
- `USB_DISCONNECTED` = 0x12
- `CALIBRATION_FAILED` = 0x20

`int auxiliary_power_get_status(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser

In [94]:
powerStatus = c_int(dgilib.auxiliary_power_get_status(power_hndl))
print("power_status: {0}".format(powerStatus.value))

power_status: 0


#### `auxiliary_power_get_calibration`
Gets the raw calibration read from the tool.

A non-zero return value indicates an error.

`int auxiliary_power_get_calibration(uint32_t power_hndl, uint8_t* data, size_t length)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*data* | Buffer that will hold the read raw calibration data
*length* | Number of raw calibration bytes to fetch. See the DGI documentation for number of bytes.

In [95]:
calibrationData = (c_ubyte * NUM_CALIBRATION)()
calibrationLength = c_size_t(NUM_CALIBRATION)
res = dgilib.auxiliary_power_get_calibration(power_hndl, byref(calibrationData), calibrationLength)
print("{0} auxiliary_power_get_calibration".format(res))
# for i in range(calibrationLength.value):
#     print("\t{0}: {1}".format(i, calibrationData[i]))

208 auxiliary_power_get_calibration


#### `auxiliary_power_register_buffer_pointers`
Registers a set of pointers to be used for storing the calibrated power data. The buffers can then be locked by auxiliary_power_lock_data_for_reading, and the data directly read from the specified buffers. Zero-pointers can be specified to get the buffers allocated within DGILib. This requires the data to be fetched using auxiliary_power_copy_data.

A non-zero return value indicates an error.

`int auxiliary_power_register_buffer_pointers(uint32_t power_hndl, float* buffer, double* timestamp, size_t* count, size_t max_count, int channel, int type)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*buffer* | Buffer that will hold the samples. Set to 0 for automatically allocated.
*timestamp* | Buffer that will hold the timestamp for the samples. Set to 0 for automatically allocated.
*count* | Pointer to a variable that will hold the count of samples. Set to 0 for automatically allocated.
*max_count* | Number of samples that can fit into the specified buffers. Or size of automatically allocated buffers.
*channel* | Power channel for this buffer: A = 0, B = 1 (Power Debugger specific)
*type* | Type of power data: Current = 0, Voltage = 1, Range = 2

In [96]:
powerBuffer = (c_float * BUFFER_SIZE)()
powerTimestamp = (c_double * BUFFER_SIZE)()
powerCount = c_size_t()
max_count = (c_size_t * BUFFER_SIZE)()
channel = c_int(0)
powerType = c_int(0)
res = dgilib.auxiliary_power_register_buffer_pointers(power_hndl, byref(powerBuffer), byref(powerTimestamp), byref(powerCount), max_count, channel, powerType);
# res = dgilib.auxiliary_power_register_buffer_pointers(power_hndl, 0, 0, 0, max_count, channel, powerType);
print("{0} auxiliary_power_register_buffer_pointers".format(res))

0 auxiliary_power_register_buffer_pointers


In [97]:
powerStatus = c_int(dgilib.auxiliary_power_get_status(power_hndl))
print("power_status: {0}".format(powerStatus.value))

power_status: 0


#### `auxiliary_power_start`
Starts parsing of power data. The power and power sync interfaces are enabled automatically, but note that it is necessary to start the polling separately. This only starts the parser that consumes data from the DGILib buffer.

A non-zero return value indicates an error.

`int auxiliary_power_start(uint32_t power_hndl, int mode, int parameter)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*mode* | Sets the mode of capture.
 | 0 - continuous capturing which requires the user to periodically consume the data.
 | 1 - oneshot capturing that captures data until the buffer has been read once, has been filled or the time from the first received sample in seconds equals the specified parameter.
*parameter* | Mode specific

In [98]:
mode = c_int(0)
parameter = c_int(0)
res = dgilib.auxiliary_power_start(power_hndl, mode, parameter);
print("{0} auxiliary_power_start".format(res))

0 auxiliary_power_start


In [100]:
powerStatus = c_int(dgilib.auxiliary_power_get_status(power_hndl))
print("power_status: {0}".format(powerStatus.value))

power_status: 1


In [101]:
#     int loggedSamples = 0;
#     printf("Waiting to log some data \n");
#     for (int i = 0; i < 25; i++) {
#         auxiliary_power_lock_data_for_reading(power_hndl);
#         auxiliary_power_copy_data(power_hndl, powerBuffer+loggedSamples, powerTimestamp+loggedSamples, &powerCount, max_count, channel, powerType);
#         auxiliary_power_free_data(power_hndl);
#         loggedSamples += powerCount;
#         printf(".%d", powerCount);
# //        printf(".");
#         for (int j = 0;j < WAIT_ITERATIONS; j+=10) {}
#     }
#     printf(" Done\n");

In [102]:
powerStatus = c_int(dgilib.auxiliary_power_get_status(power_hndl))
print("power_status: {0}".format(powerStatus.value))

power_status: 1


In [103]:
#     interface_id = INTERFACE_GPIO;
#     res = interface_read_data(dgi_hndl, interface_id, readBuffer, timestamp, &length, &ovf_index, &ovf_length, &ovf_entry_count);
#     printf("%d interface_read_data: 0x%.2x, length: %d\n", res, interface_id, length);
#     for (int i = 0; i < length; ++i) {
#         printf("\t%.6d: buffer: %.4u, timestamp: %.4llu, time: %lf\n", i, readBuffer[i], timestamp[i], timestamp[i] * timerFactor);
#     }

#### `auxiliary_power_lock_data_for_reading`
Blocks the parsing thread from accessing all the buffers. This must be called before the user application code accesses the buffers, or a call to auxiliary_power_copy_data is made. Afterwards auxiliary_power_free_data must be called. Minimize the amount of time between locking and freeing to avoid buffer overflows.

A non-zero return value indicates an error.

`int auxiliary_power_lock_data_for_reading(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser

In [104]:
res = dgilib.auxiliary_power_lock_data_for_reading(power_hndl)
print("{0} auxiliary_power_lock_data_for_reading".format(res))

0 auxiliary_power_lock_data_for_reading


#### `auxiliary_power_copy_data`
Copies parsed power data into the specified buffer. Remember to lock the buffers first. If the count parameter is the same as max_count there is probably more data to be read. Do another read to get the remaining data.

A non-zero return value indicates an error.

`int auxiliary_power_copy_data(uint32_t power_hndl, float* buffer, double* timestamp, size_t* count, size_t
max_count, int channel, int type)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*buffer* | Buffer that will hold the data
*timestamp* | Buffer that will hold the timestamps
*count* | Pointer to a variable that will hold the count of elements copied
*max_count* | Maximum number of elements that the buffer can hold
*channel* | Power channel for this buffer: A = 0, B = 1 (Power Debugger specific)
*type* | Type of power data: Current = 0, Voltage = 1, Range = 2

In [105]:
res = dgilib.auxiliary_power_copy_data(power_hndl, powerBuffer, powerTimestamp, byref(powerCount), max_count, channel, powerType);
print("{0} auxiliary_power_copy_data: {1} samples".format(res, powerCount.value))

0 auxiliary_power_copy_data: 0 samples


#### `auxiliary_power_free_data`
Clears the power data buffers and allows the power parser to continue.

A non-zero return value indicates an error.

`int auxiliary_power_free_data(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser

In [106]:
res = dgilib.auxiliary_power_free_data(power_hndl)
print("{0} auxiliary_power_free_data".format(res))

0 auxiliary_power_free_data


#### `auxiliary_power_stop`
Stops parsing of power data.

A non-zero return value indicates an error.

`int auxiliary_power_stop(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser

In [107]:
res = dgilib.auxiliary_power_stop(power_hndl)
print("{0} auxiliary_power_stop".format(res))

0 auxiliary_power_stop


In [ ]:
# print("captured power data:")
# for i in range(powerCount.value):
#     print("\t{0}: buffer: {1}, timestamp: {2}".format(i, powerBuffer[i], powerTimestamp[i]))

In [113]:
powerStatus = c_int(dgilib.auxiliary_power_get_status(power_hndl))
print("power_status: {0}".format(powerStatus.value))

power_status: 0


In [114]:
#     interface_id = INTERFACE_GPIO;
#     res = interface_read_data(dgi_hndl, interface_id, readBuffer, timestamp, &length, &ovf_index, &ovf_length, &ovf_entry_count);
#     printf("%d interface_read_data: 0x%.2x, length: %d\n", res, interface_id, length);
#     for (int i = 0; i < length; ++i) {
#         printf("\t%.6d: buffer: %.4u, timestamp: %.4llu, time: %lf\n", i, readBuffer[i], timestamp[i], timestamp[i] * timerFactor);
#     }

In [115]:
#     res = interface_read_data(dgi_hndl, INTERFACE_POWER_SYNC, readBuffer, timestamp, &length, &ovf_index, &ovf_length, &ovf_entry_count);
#     printf("%d interface_read_data: 0x%.2x, length: %d\n", res, INTERFACE_POWER_SYNC, length);
#     for (int i = 0; i < length; ++i) {
#         printf("\t%.6d: buffer: %.4u, timestamp: %.4llu, time: %lf\n", i, readBuffer[i], timestamp[i], timestamp[i] * timerFactor);
#     }

## Uninitialize

Here are all the functions to shutdown the device gracefully and avoid the dreaded mode 17.

#### `auxiliary_power_unregister_buffer_pointers`
Unregisters the pointers for the specified power channel.

A non-zero return value indicates an error.

`int auxiliary_power_unregister_buffer_pointers(uint32_t power_hndl, int channel, int type)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*channel* | Power channel for this buffer: A = 0, B = 1 (Power Debugger specific)
*type* | Type of power data: Current = 0, Voltage = 1, Range = 2

In [116]:
res = dgilib.auxiliary_power_unregister_buffer_pointers(power_hndl, channel, powerType)
print("{0} auxiliary_power_unregister_buffer_pointers".format(res))

0 auxiliary_power_unregister_buffer_pointers


#### `auxiliary_power_uninitialize`
Uninitializes the power parser.

A non-zero return value indicates an error.

`int auxiliary_power_uninitialize(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser

In [117]:
res = dgilib.auxiliary_power_uninitialize(power_hndl)
print("{0} auxiliary_power_uninitialize".format(res))

0 auxiliary_power_uninitialize


#### `stop_polling`
This function will stop the polling system and stop acquisition on all interfaces.

A non-zero return value indicates an error.

`int stop_polling(uint32_t dgi_hndl)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle of the connection

In [118]:
res = dgilib.stop_polling(dgi_hndl)
print("{0} stop_polling".format(res))

0 stop_polling


#### `interface_disable`
Disables the specified interface.

A non-zero return value indicates an error.

`int interface_disable(uint32_t dgi_hndl, int interface_id)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface to enable

In [119]:
interface_id = INTERFACE_GPIO
res = dgilib.interface_disable(dgi_hndl, interface_id)
print("{0} interface_disable: {1}".format(res, interface_id))

#     res = interface_disable(dgi_hndl, interface_id);
#     printf("%d interface_disable: 0x%.2x\n", res, interface_id);

#     res = interface_disable(dgi_hndl, INTERFACE_POWER_SYNC);
#     printf("%d interface_disable: 0x%.2x\n", res, INTERFACE_POWER_SYNC);

#     res = interface_disable(dgi_hndl, INTERFACE_TIMESTAMP);
#     printf("%d interface_disable: 0x%.2x\n", res, INTERFACE_TIMESTAMP);

0 interface_disable: 48


#### `disconnect`
Closes the specified connection.

A non-zero return value indicates an error.

`int disconnect(uint32_t dgi_hndl)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle of the connection

In [120]:
res = dgilib.disconnect(dgi_hndl)
print("{0} disconnect".format(res))

0 disconnect


In [121]:
c_status = dgilib.connection_status(dgi_hndl)
print("connection_status: {0}".format(c_status))

connection_status: 2


In [122]:
dgilib.UnInitialize(dgi_hndl)

2

In [123]:
def foo(aqs : typing.List[int]) -> Array:
    array_type = c_int64 * len(aqs)
    ans = array_type(*aqs)
    return ans

for el in foo([1,2,3]):
    print(el)

1
2
3
